In [1]:
import neo4j

host = "localhost"
username = "neo4j"
password = "1234qwer"


driver = neo4j.GraphDatabase.driver(f'bolt://localhost:7687', auth=(username, password))
session = driver.session()

def create_graph(query):
    session.run(query)

In [2]:
from sentence_transformers import SentenceTransformer

def sentence_transformer_embedding(text):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embeddings = model.encode(text)
    return embeddings

/Users/fahmisyaifudin/miniconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [4]:
session.run("""
MATCH (p:Requirement)
RETURN gds.graph.project(
  'requirementGraph',
  p,
  null,
  {
    sourceNodeProperties: p { .sentence_embedding },
    targetNodeProperties: {}
  }
)
""")

In [5]:
import pandas as pd

entities = []
records, summary, key = driver.execute_query("""
CALL gds.knn.stream('requirementGraph', {
    topK: 2,
    nodeProperties: { sentence_embedding: 'EUCLIDEAN'},
    randomSeed: 1337,
    concurrency: 1,
    sampleRate: 1.0,
    deltaThreshold: 0.0
})
YIELD node1, node2, similarity
WHERE gds.util.asNode(node1).app <> gds.util.asNode(node2).app
RETURN gds.util.asNode(node1).app AS app1, gds.util.asNode(node1).sentence AS Req1, gds.util.asNode(node2).app AS app2, gds.util.asNode(node2).sentence AS Req2, similarity
ORDER BY similarity DESC
""")

for record in records:
    entities.append({"app1": record['app1'], "entity1": record['Req1'], "app2": record['app2'], "entity2": record['Req2'], "similarity": record['similarity']})
df_entity = pd.DataFrame(entities)

In [6]:
df_entity[:10]

,app1,entity1,app2,entity2,similarity
0,Odoo Purchase,Use predefined dashboards or build your own wi...,Odoo CRM,Use predefined dashboards or build your own wi...,0.894105
1,Odoo CRM,Use predefined dashboards or build your own wi...,Odoo Purchase,Use predefined dashboards or build your own wi...,0.894105
2,Odoo Sales,Add product variants to your sales orders usin...,Odoo Purchase,Add product variants to your purchase orders w...,0.845759
3,Odoo Purchase,Add product variants to your purchase orders w...,Odoo Sales,Add product variants to your sales orders usin...,0.845759
4,Odoo Sales,Follow key quotations and business opportuniti...,Odoo CRM,Follow key opportunities in just a click and g...,0.746597
5,Odoo CRM,Follow key opportunities in just a click and g...,Odoo Sales,Follow key quotations and business opportuniti...,0.746597
6,Softeng POS,The system must update the backend inventory s...,SCATS,"After the sale is completed, the system must u...",0.726185
7,SCATS,"After the sale is completed, the system must u...",Softeng POS,The system must update the backend inventory s...,0.726185
8,Odoo Sales,Compute shipping costs based on multiple varia...,Odoo Inventory,Integrate with third-party shipping carriers t...,0.725947
9,Odoo Inventory,Integrate with third-party shipping carriers t...,Odoo Sales,Compute shipping costs based on multiple varia...,0.725947
